<a href="https://colab.research.google.com/github/kimcaprio/lifeofkaggle/blob/master/ch02_santander/Baseline_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baseline 모델
## tabular 데이터를 다루는 캐클의 파이프라인
### 1. 데이터 전처리 -> 2. 피처엔지니어링 -> 3. 머신러닝 모델학습 -> 4. 테스트 데이터 예측 및 캐글 업로드


1.   데이터 전처리

*   제품 변수의 결측값을 0으로 대체. 제품 보유 여부에 대한 정보가 없으면, 해당 제품을 보유하지 않고 있지 않다고 가정.
*   훈련 데이터와 테스트 데이터를 통합. 훈련 데이터와 테스트 데이터는 날짜변수(fecha_dato)로 쉽게 구분 가능. 동일한 24개의 고객 변수를 공유하고 있으며, 테스트 데이터에 없는 24개의 제품 변수는 0으로 채움.
* 번주형, 수치형 데이터를 전처리. 변수형 데이터는 .factorize()통해 label encoding을 수행. 데이터 타입이 object로 표현되는 수치형 데이터에서는 .unique()를 통해 특이값들을 대체하거나 제거하고, 정수형 데이터로 변환.
*   추후 모델 학습에 사용할 변수 이름을 features 리스트에 미리 담는다.







In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/My Drive/CoLab/explorer_of_machine_learning/for_kaggle/.kaggle/" # put path for wherever you put it

In [0]:
import pandas as pd
import numpy as np
import xgboost as xgb

np.random.seed(2018)

# 데이터를 불러온다
trn = pd.read_csv('/content/drive/My Drive/CoLab/explorer_of_machine_learning/ch02_santander/data/train_ver2.csv')
tst = pd.read_csv('/content/drive/My Drive/CoLab/explorer_of_machine_learning/ch02_santander/data/test_ver2.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
## 데이터 전처리
# 제품 변수를 별도로 저장
prods = trn.columns[24:].tolist()

# 제품 변수 결측값을 미리 0으로 대체
trn[prods] = trn[prods].fillna(0.0).astype(np.int8)

# 24개 제품 중 하나도 보유하지 않는 고객 데이터를 제거
no_product = trn[prods].sum(axis=1) == 0
trn = trn[-no_product]

# 훈련 데이터와 테스트 데이터를 통합. 테스트 데이터에 없는 제품 변수는 0으로 대체
for col in trn.columns[24:]:
  tst[col] = 0
df = pd.concat([trn, tst], axis=0)

# 학습에 사용할 변수를 담는 list
features = []

# 범주형 변수를 .factorize() 함수를 통해 label encoding 함(one-hot)
categorical_cols = [col for col in trn.columns[:24] if trn[col].dtype in ['O']]
print(categorical_cols)
for col in categorical_cols:
  df[col], _ = df[col].factorize(na_sentinel=-99)
features += categorical_cols

# 수치형 변수의 특이값과 결측값을 -99로 대체하고, 정수형으로 변환
df['age'].replace('NA', -99, inplace=True)
df['age'] = df['age'].astype(np.int8)

df['antiguedad'].replace('  NA', -99, inplace=True)
df['antiguedad'] = df['antiguedad'].astype(np.int8)

df['renta'].replace('   NA', -99, inplace=True)
df['renta'].fillna(-99, inplcate=True)
df['renta'] = df['renta'].astype(float).astype(np.int8)

df['indrel_1mes'].replace('P', 5, inplace=True)
df['indrel_1mes'].fillna(-99, inplcate=True)
df['indrel_1mes'] = df['indrel_1mes'].astype(float).astype(np.int8)

#학습에 사용할 수치형 변수를 features에 추가
features =  [col for col in trn.columns[:24] if trn[col].dtype in ['int64', 'float64']]
print(features)

['fecha_dato', 'ind_empleado', 'pais_residencia', 'sexo', 'age', 'fecha_alta', 'antiguedad', 'ult_fec_cli_1t', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp', 'canal_entrada', 'indfall', 'nomprov', 'segmento']


TypeError: ignored

2.   피처 엔지니어링
*   모델 학습에 사용할 파생변수를 생성.

In [0]:
# (피처 엔지니어링) 두 날짜 변수에서 연도와 월 정보를 추출
df['fecha_alta_month'] = df['fech_dato'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)